In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#------------------------------------------------------------
# Load Data
#------------------------------------------------------------

people = pd.read_csv('/Users/gurumac/Downloads/archive/01_people.csv')
education = pd.read_csv('/Users/gurumac/Downloads/archive/03_education.csv')
experience = pd.read_csv('/Users/gurumac/Downloads/archive/04_experience.csv')
person_skills = pd.read_csv('/Users/gurumac/Downloads/archive/05_person_skills.csv')
skills = pd.read_csv('/Users/gurumac/Downloads/archive/06_skills.csv')




In [ ]:
#------------------------------------------------------------
# Data Normalization and Preprocessing
#------------------------------------------------------------

# 1. Normalize the job titles
experience.loc[:, 'title'] = (
    experience['title']
    .str.lower()
    .str.replace(r'[^\w\s]', '', regex=True)
    .str.strip()
)

# Function to normalize date formats so that they can be ordered
def normalize_date(date_str):
    if pd.isnull(date_str) or date_str.lower() == "present":
        return 999999  # Present
    match = re.match(r'(\d{1,2})/(\d{2,4})', date_str)
    if match:
        month, year = match.groups()
        year = year if len(year) == 4 else f"19{year}"  # Assume 1900s for 2-digit years
        return int(year + month.zfill(2))  # Format as YYYYMM
    return np.nan

# Normalize dates
experience['start_ym'] = experience['start_date'].apply(normalize_date)
experience['end_ym'] = experience['end_date'].apply(normalize_date)

# Sort by person_id and latest end date, fallback to start date if end_date is missing
experience_sorted = experience.sort_values(
    by=['person_id', 'end_ym', 'start_ym'], ascending=[True, False, False]
)

# Pick the most recent job per person
recent_experience = experience_sorted.drop_duplicates(subset=['person_id'], keep='first')

# Merge with the people table
df = pd.merge(people, recent_experience[['person_id', 'title']], on='person_id', how='left')

In [4]:
# 3. Aggregate skills per person
person_skills_full = pd.merge(person_skills, skills, on='skill', how='left')
# person_skills_full['skill'] =  skill name

# Clean skill names same way we did title
def clean_skill_name(skill):
    if pd.isna(skill):
        return None  # Return None if skill is NaN
    skill = skill.lower()
    skill = re.sub(r'[^\w\s]', '', skill)
    skill = skill.strip()
    if len(skill) < 2:  # Remove very short or irrelevant skills
        return None
    return skill

person_skills_full['cleaned_skill'] = person_skills_full['skill'].apply(clean_skill_name)
person_skills_full = person_skills_full.dropna(subset=['cleaned_skill'])

# Group skills by person_id
person_skills_agg = person_skills_full.groupby('person_id')['cleaned_skill'].apply(list).reset_index(name='skill_list')

# Merge skill lists into the main df
df = pd.merge(df, person_skills_agg, on='person_id', how='left')

# For individuals with no skills, assign empty lists
df['skill_list'] = df['skill_list'].apply(lambda x: x if isinstance(x, list) else [])

# 4. Process Education (program column)
# pick the highest-level CS-related degree
education['program'] = education['program'].fillna('').astype(str)
education.loc[:, 'program'] = (
    education['program']
    .str.lower()
    .str.replace(r'[^\w\s]', '', regex=True)
    .str.strip()
)

def map_program(p):
    # Identify CS-related program and degree level
    is_cs = 'computer' in p
    # Determine level
    if 'associate' in p:
        level = 1
    elif 'bachelor' in p:
        level = 2
    elif 'master' in p:
        level = 3
    elif 'phd' in p or 'doctor' in p:
        level = 4
    else:
        level = 0
    # Return level if CS related, else 0
    return level if is_cs else 0

education['program_level'] = education['program'].apply(map_program)

# For each person, pick the highest CS program level
highest_program = education.groupby('person_id')['program_level'].max().reset_index()
df = pd.merge(df, highest_program, on='person_id', how='left')
df['program_level'] = df['program_level'].fillna(0)


In [14]:


#------------------------------------------------------------
# Create Target Variable (CS vs Non-CS)
#------------------------------------------------------------
cs_keywords = [
    'software engineer', 'data scientist', 'data analyst', 'machine learning engineer',
    'devops engineer', 'backend developer', 'frontend developer', 'full stack developer',
    'systems engineer', 'database administrator', 'cloud engineer', 'computer vision',
    'ai engineer', 'artificial intelligence', 'robotics engineer', 'ml engineer'
]

def is_cs_role(title):
    if pd.isnull(title):
        return 0
    title_lower = title.lower()
    return 1 if any(kw in title_lower for kw in cs_keywords) else 0

df['cs_label'] = df['title'].apply(is_cs_role)


In [5]:


from collections import Counter

# Flatten the list of all skills and count each occurrence
all_skills = [skill for sublist in df['skill_list'] for skill in sublist]
skill_counts = Counter(all_skills)

# Filter skills by setting a minimum frequency threshold, e.g., skills must appear more than 100 times
min_frequency = 100
filtered_skills = {skill for skill, count in skill_counts.items() if count > min_frequency}

# Apply filter to the skill lists in the DataFrame
df['filtered_skill_list'] = df['skill_list'].apply(lambda skills: [s for s in skills if s in filtered_skills])


unique_skills = set(x for lst in df['filtered_skill_list'] for x in lst)
print("Number of unique skills:", len(unique_skills))



Number of unique skills: 2043


In [8]:

#------------------------------------------------------------
# Skills Encoding with MultiLabelBinarizer
#------------------------------------------------------------
mlb = MultiLabelBinarizer()
skill_matrix = mlb.fit_transform(df['filtered_skill_list'])

skill_features = pd.DataFrame(skill_matrix, columns=mlb.classes_, index=df.index)

# Optional: Dimensionality Reduction on Skills
svd = TruncatedSVD(n_components=50, random_state=42)
skill_reduced = svd.fit_transform(skill_features)


In [16]:
import pandas as pd

# Display general info about the DataFrame
print("DataFrame Information:")
df.info()

# Show the first few rows of the DataFrame to understand the data format
print("\nFirst few rows of the DataFrame:")
print(df.head())

# Check for missing values in the DataFrame
print("\nMissing Values in Each Column:")
print(df.isnull().sum())

# Display unique values and their counts for the target variable 'cs_label'
print("\nDistribution of Target Variable 'cs_label':")
print(df['cs_label'].value_counts())

# Ensure the skill matrix is correctly merged (if using MultiLabelBinarizer output directly)
if 'skill_matrix' in locals() or 'skill_matrix' in globals():
    print("\nFirst few rows of the Skill Matrix:")
    print(pd.DataFrame(skill_matrix, columns=mlb.classes_).head())

# Check dimensionality of the feature matrix if it has been reduced
if 'skill_reduced' in locals() or 'skill_reduced' in globals():
    print("\nShape of Reduced Skill Features:")
    print(skill_reduced.shape)

# Print out the shape of the final feature set 'X' and target 'y'
if 'X' in locals() or 'X' in globals() and 'y' in locals() or 'y' in globals():
    print("\nShape of Feature Matrix 'X' and Target Array 'y':")
    print("X:", X.shape, "y:", y.shape)

# Verify that all features are numeric (required by Random Forest and most other algorithms)
if 'X' in locals() or 'X' in globals():
    print("\nData Types of Features in 'X':")
    print(pd.DataFrame(X).dtypes.value_counts())

# Additional checks (optional): View summary statistics for numeric columns
print("\nSummary Statistics for Numeric Columns:")
print(df.describe())


DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54933 entries, 0 to 54932
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person_id            54933 non-null  int64  
 1   name                 54819 non-null  object 
 2   email                1593 non-null   object 
 3   phone                1833 non-null   object 
 4   linkedin             8538 non-null   object 
 5   title                54927 non-null  object 
 6   skill_list           54933 non-null  object 
 7   program_level        54933 non-null  float64
 8   filtered_skill_list  54933 non-null  object 
 9   cs_label             54933 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 4.2+ MB

First few rows of the DataFrame:
   person_id                                               name email phone  \
0          1                             Database Administrator   NaN   NaN   
1          2  

In [18]:
#------------------------------------------------------------
# Final Feature Assembly
#------------------------------------------------------------
X = np.hstack([
    skill_reduced, 
    df[['program_level']].fillna(0).values
])
y = df['cs_label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [20]:
#------------------------------------------------------------
# Model Training
#------------------------------------------------------------
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print("Model Accuracy:", score)

#------------------------------------------------------------
# Skill Importance (without SVD for interpretability)
#------------------------------------------------------------
# Retrain a model directly on skill_features (no SVD) to interpret skill importance
model_no_svd = RandomForestClassifier(n_estimators=100, random_state=42)
model_no_svd.fit(skill_features, y)

importances = model_no_svd.feature_importances_
feature_importances = pd.Series(importances, index=skill_features.columns).sort_values(ascending=False)

print("Top 20 important skills:\n", feature_importances.head(20))



# STRATIFIED KFOLDS

Model Accuracy: 0.9831619186311095
Top 20 important skills:
 rman                       0.022340
performance tuning         0.015855
dba                        0.012709
replication                0.009823
database security          0.008597
sql dba                    0.008589
data guard                 0.008558
database                   0.008408
oracle dba                 0.007022
ssis                       0.006325
log shipping               0.006321
database administration    0.006124
sql server                 0.006081
rac                        0.005810
javascript                 0.005573
sql                        0.005493
ssrs                       0.005266
mysql                      0.005219
html                       0.005114
database migration         0.004988
dtype: float64


In [22]:
#------------------------------------------------------------
# Skills Gap Analysis
#------------------------------------------------------------
possession_rate = skill_features.mean() * 100
gap_analysis = pd.DataFrame({
    'importance': feature_importances,
    'possession_rate': possession_rate
})

gap_analysis['gap_score'] = gap_analysis['importance'] * (100 - gap_analysis['possession_rate'])
gap_analysis = gap_analysis.sort_values('gap_score', ascending=False)
print("Top gap skills:\n", gap_analysis.head(20))

Top gap skills:
                          importance  possession_rate  gap_score
rman                       0.022340         2.288242   2.182905
performance tuning         0.015855         3.396865   1.531628
dba                        0.012709         0.933865   1.259035
replication                0.009823         1.594670   0.966676
sql dba                    0.008589         0.622577   0.853504
database security          0.008597         1.583747   0.846105
data guard                 0.008558         1.294304   0.844737
database                   0.008408         2.856207   0.816749
oracle dba                 0.007022         0.628038   0.697769
log shipping               0.006321         0.802796   0.627069
ssis                       0.006325         2.009721   0.619765
database administration    0.006124         3.500628   0.590993
rac                        0.005810         1.256076   0.573732
sql server                 0.006081         7.405385   0.563056
ssrs                   